In [9]:
from egoexo_dataloader_2 import EgoExoDataset
import os
import numpy as np
import torch
import random
from torch.utils.data import DataLoader

from mmpose.apis import MMPoseInferencer
from mmaction.apis import pose_inference, detection_inference

/home/thibault/miniconda3/envs/ego-challenge/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/thibault/miniconda3/envs/ego-challenge/lib/python3.12/site-packages/mmengine/optim/optimizer/zero_optimizer.py:11: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import \


In [10]:
dataset_path = '/media/thibault/T5 EVO/Datasets/Ego4D/'
train_dataset = EgoExoDataset(dataset_path, os.path.join(dataset_path, 'takes.json'), split = "train", skill=True, get_frames=False, get_pose=True, get_hands_pose=False, frame_rate=3, transform=None)
val_dataset = EgoExoDataset(dataset_path, os.path.join(dataset_path, 'takes.json'), split = "val", skill=True, get_frames=False, get_pose=True, get_hands_pose=False, frame_rate=3, transform=None)


In [11]:
data = train_dataset.__getitem__(50)
data["pose"]["0"][0]

{'keypoints': array([[[ 324.78924298,  109.18517376],
         [ 239.48237092,    0.        ],
         [ 239.14007637,    0.        ],
         [ 189.68002439,  305.32573508],
         [ 674.81190048,    0.        ],
         [   0.        ,    2.65967957],
         [   0.        , 1596.07089956],
         [   0.        ,    0.        ],
         [1895.65166719,    0.        ],
         [ 724.19771439,  524.21631527],
         [ 772.65556683,    0.        ],
         [2273.14422583,  896.69795007],
         [   0.        , 1194.04559673],
         [ 638.85083048,  354.18346093],
         [1246.29705644,  109.61015297],
         [1567.68725515,  291.61553776],
         [1240.30220035,  161.4711899 ]]]),
 'keypoint_scores': array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5,
        0.5, 0.5, 0.5, 0.5])}

In [12]:
paths = [path for path in data["samples"]["exo"]]
paths.append(data["samples"]["ego"])
paths

['/media/thibault/T5 EVO/Datasets/Ego4D/takes/sfu_basketball_02_6/frame_aligned_videos/downscaled/448/cam01.mp4',
 '/media/thibault/T5 EVO/Datasets/Ego4D/takes/sfu_basketball_02_6/frame_aligned_videos/downscaled/448/cam02.mp4',
 '/media/thibault/T5 EVO/Datasets/Ego4D/takes/sfu_basketball_02_6/frame_aligned_videos/downscaled/448/cam03.mp4',
 '/media/thibault/T5 EVO/Datasets/Ego4D/takes/sfu_basketball_02_6/frame_aligned_videos/downscaled/448/cam04.mp4',
 '/media/thibault/T5 EVO/Datasets/Ego4D/takes/sfu_basketball_02_6/frame_aligned_videos/downscaled/448/aria01_214-1.mp4']

In [13]:
print(len(train_dataset), len(val_dataset))

562 148


In [14]:
skeletons = list(data["pose"].values())


for skeleton in skeletons:
    for frame in skeleton:
        print(np.min(frame["keypoints"]))
        

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
806.9889326974848
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
977.6647086392643
976.1833602899011
980.8849871969322
0.0
998.601866491738
999.2158072104893
998.6561436384526
1000.7538677670885
1002.7038410535731
1015.3759564976514
1018.2303965667264
1020.0368429834709
1019.0006538816368
1017.6920906708452
1018.2766597913944
1024.0568736550058
1027.3004150391
1026.571852340533
1021.9946134016747
1025.9010558108273
1029.8149573991739
1034.3537228769453
0.0
1032.8347540219404
0.0
1035.7521972656
1033.3742675781
1044.0394287109
0.0
1046.8957519531
0.0
1049.1307373047
1054.6950683594
1056.0291748047
1055.2081298828
1053.69

In [15]:
from mmaction.apis import inference_skeleton, init_recognizer
from mmengine import Config
import torch
torch.cuda.empty_cache()
config_path = "/home/thibault/Documents/Code/pckg/mmaction2/configs/skeleton/posec3d/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint.py"
checkpoint_path = "/home/thibault/Documents/Code/pckg/mmaction2/checkpoints/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint_20220815-38db104b.pth" # can be a local path   # you can specify your own picture path

cfg = Config.fromfile(config_path)



# build the model from a config file and a checkpoint file
skeleton_model = init_recognizer(cfg, checkpoint_path, device="cuda:0")


Loads checkpoint by local backend from path: /home/thibault/Documents/Code/pckg/mmaction2/checkpoints/slowonly_r50_8xb16-u48-240e_ntu60-xsub-keypoint_20220815-38db104b.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: cls_head.fc_cls.weight, cls_head.fc_cls.bias



/home/thibault/miniconda3/envs/ego-challenge/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/thibault/miniconda3/envs/ego-challenge/lib/python3.12/site-packages/mmengine/runner/checkpoint.py:347: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they 

In [16]:
import torch

def collate_pose_and_label_with_padding(batch):
    """
    batch: list of samples, each a dict with:
      - 'pose':  np.ndarray (dtype=object) length N_i of view‐dicts
      - 'label': int

    Returns:
      padded_keypoints: FloatTensor (B, V_max, K, 2)
      padded_scores:    FloatTensor (B, V_max, K)
      view_mask:        BoolTensor  (B, V_max)
      labels:           LongTensor  (B,)
    """
    B = len(batch)
    # 1) labels
    labels = torch.tensor([s['label'] for s in batch], dtype=torch.long)

    # 2) collect pose-arrays and find max views
    pose_arrays = [s['pose'] for s in batch]            # list of np.object arrays
    num_views   = [len(pa) for pa in pose_arrays]       # N_i
    V_max       = max(num_views)

    # 3) get joint-count K (and dim=2) from the first view of first sample
    sample0_view0 = pose_arrays[0]['0'][0]
    _, K, dim = sample0_view0['keypoints'].shape        # (1, K, 2)

    # 4) allocate padded tensors
    padded_keypoints = torch.zeros(B, V_max, K, dim, dtype=torch.float)
    padded_scores    = torch.zeros(B, V_max, K,    dtype=torch.float)
    view_mask        = torch.zeros(B, V_max,        dtype=torch.bool)

    # 5) fill in real views
    for b, pa in enumerate(pose_arrays):
        for v, view_dict in enumerate(pa.values()):
            # squeeze out that leading 1 if present
            kp = view_dict['keypoints']
            if kp.ndim == 3 and kp.shape[0] == 1:
                kp = kp[0]            # now (K, 2)
            scores = view_dict['keypoint_scores']  # (K,)

            padded_keypoints[b, v] = torch.from_numpy(kp)
            padded_scores[b, v]    = torch.from_numpy(scores)
            view_mask[b, v]        = True

    return padded_keypoints, padded_scores, view_mask, labels


In [17]:
import torch

def collate_pose_and_label_with_padding_2(batch):
    """
    batch: list of samples, each a dict with:kp
      - 'pose':  np.ndarray (dtype=object) length N_i of view‐dicts
      - 'label': int, V_max)
      labels:           LongTensor  (B,)
    """
    B = len(batch)
    # 1) labels
    labels = torch.tensor([s['skill'] for s in batch], dtype=torch.long)

    # 2) collect pose-arrays and find max views
    pose_arrays = [s['pose'] for s in batch]            # list of np.object arrays
    num_views   = [len(pa) for pa in pose_arrays]       # N_i
    V_max       = max(num_views)

    for sample in batch:
      if len(sample["pose"].keys()) < V_max:
        for i in range(len(sample["pose"].keys()), V_max):
          T = len(sample["pose"]["0"])
          sample["pose"][str(i)] = np.array(np.repeat({"keypoints": np.zeros((V_max, 17, 2)), "keypoint_scores": np.zeros((V_max, 17))}, T))
       
        
      
    
    return pose_arrays, labels


In [18]:
import torch

def collate_objects(batch):
    """
    batch: list of samples, each a dict with:kp
      - 'pose':  np.ndarray (dtype=object) length N_i of view‐dicts
      - 'label': int, V_max)
      labels:           LongTensor  (B,)
    """
    B = len(batch)
    # 1) labels
    labels = torch.tensor([s['skill'] for s in batch], dtype=torch.long)

    # 2) collect pose-arrays and find max views
    pose_arrays = [s for s in batch]            # list of np.object arrays
        
      
    
    return pose_arrays, labels


In [19]:
from pathlib import Path
DETECTION_CONFIG = Path(
    "./generate_skeletons/"
    "faster-rcnn_r50_fpn_2x_coco_infer.py"
)
DETECTION_CHECKPOINT = Path(
    "./generate_skeletons/"
    "faster_rcnn_r50_fpn_2x_coco_bbox_mAP-0.384_20200504_210434-a5d8aa15.pth"
)

SKELETON_CONFIG = Path(
    "./generate_skeletons/"
    "td-hm_hrnet-w32_8xb64-210e_coco-256x192_infer.py"
)
SKELETON_CHECKPOINT = Path(
    "./generate_skeletons/"
    "hrnet_w32_coco_256x192-c78dce93_20200708.pth"
)

WINDOW_SIZE = 128
SAMPLING_RATE = 3

In [20]:

import cv2
import numpy as np
from typing import Callable, Generator, List
from IPython.display import clear_output
import os
import contextlib


def frame_windows(video_path: str,
                  window_size: int,
                  sampling_rate: int = 1,
                  num_windows: int = 1,
                 ) -> Generator[List[np.ndarray], None, None]:
    """
    Lazily read frames from the video, sampling 1 in every `sampling_rate`,
    and yield lists of up to `window_size` sampled frames.
    """
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f'Length of video: {total_frames}')
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return None
    stride = (total_frames - window_size * sampling_rate) // max(num_windows - 1, 1)
    
    for i in range(num_windows):
        start_idx = i * stride
        frames: List[np.ndarray] = []

        for j in range(window_size):
            frame_idx = start_idx + j * sampling_rate
            if frame_idx >= total_frames:
                break  # Avoid reading past the video

            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
            ret, frame = cap.read()
            if not ret:
                break
            frames.append(frame)

        if len(frames) == window_size:
            yield frames

    cap.release()

def extract_skeleton(video_path: str,
                     window_size: int = 32,
                     sampling_rate: int = 1,
                     num_windows: int = 1,
                     predict: Callable[[List[np.ndarray]], np.ndarray] = None
                    ) -> np.ndarray:
    """
    Process a video in non-overlapping windows to extract per-frame skeletons
    without ever loading the whole video into RAM at once.

    Args:
        video_path: Path to the input video file.
        window_size: Number of frames per chunk.
        predict: Function(frames_list) → np.ndarray of shape
                 (num_frames, num_keypoints, dims).

    Returns:
        skeleton_sequence: np.ndarray of shape
            (total_frames, num_keypoints, dims).
    """
    if predict is None:
        raise ValueError("You must pass in a `predict` function.")
    
    skeleton_chunks = []
    total = 0
    for frames in frame_windows(video_path, window_size, sampling_rate, num_windows):
        # frames is at most window_size in length

# Suppose this is the noisy function:
# from some_lib import noisy_function

        with open(os.devnull, 'w') as devnull, \
            contextlib.redirect_stdout(devnull), \
            contextlib.redirect_stderr(devnull):
            # Anything printed to stdout or stderr inside here is discarded
            

            det_results, _ = detection_inference(str(DETECTION_CONFIG), str(DETECTION_CHECKPOINT), frames)
            pose_results, pose_data_samples = pose_inference(str(SKELETON_CONFIG), str(SKELETON_CHECKPOINT), frames, det_results)
            # e.g. (len(frames), K, D)
            skeleton_chunks.append(np.array(pose_results))
            total += len(pose_results)
        print(total)
    
    # Concatenate along temporal axis:
    if len(skeleton_chunks) == 0:
        skeleton_sequence = np.array([])
    else:
        skeleton_sequence = np.concatenate(skeleton_chunks, axis=0)
    assert skeleton_sequence.shape[0] == total

    return skeleton_sequence


In [21]:
test_skeleton = np.load("test_skeleton.npy", allow_pickle=True)
test_skeleton

array([[{'keypoints': array([[[188.83664 ,  58.62571 ],
                [188.83664 ,  52.88029 ],
                [183.09122 ,  52.88029 ],
                [185.96393 ,  55.753002],
                [162.98225 ,  52.88029 ],
                [177.34581 ,  87.3528  ],
                [148.61871 ,  87.3528  ],
                [194.58206 , 121.82531 ],
                [174.4731  , 127.570724],
                [226.18185 , 104.58906 ],
                [223.30914 , 107.46176 ],
                [180.2185  , 176.40678 ],
                [160.10954 , 179.2795  ],
                [180.2185  , 233.86096 ],
                [168.72768 , 239.60638 ],
                [177.34581 , 276.9516  ],
                [180.2185  , 305.67868 ]],

               [[483.9736  , 116.81264 ],
                [493.04993 , 110.76175 ],
                [483.9736  , 110.76175 ],
                [526.3298  , 116.81264 ],
                [523.3044  , 113.78719 ],
                [544.4825  , 177.3215  ],
                [5

In [22]:
def process_skeletons(poses, skeleton_model, pose_inference, window_size=128, sampling_rate=3, num_windows=1):
    """
    Process a batch of skeletons.
    Args:
        poses: list of dicts with keys "ego" and "exo"
        pose_inference: function to extract skeletons
    Returns:
        batch: list of tensors with shape (B, T, D)
    """
    skeletons_batch = []
    for data in poses:
        paths = [path for path in data["samples"]["exo"]]
        paths.append(data["samples"]["ego"])
        
        skeletons = {}
        
        for i, path in enumerate(paths):
            # Extract skeletons from the video
            skeletons[str(i)] = extract_skeleton(path, window_size, sampling_rate, num_windows, predict=pose_inference)
        skeletons_batch.append(skeletons)
    batch = []
    
    for sample in skeletons_batch:
        views = []
        for view in sample.values():
            representation = inference_skeleton(skeleton_model, view,(1920,1080), test_pipeline=None)  # (B, T, D)
            views.append(representation)  # (B, T, D)
        batch.append(torch.stack(views))  # (B, T, D)
    return batch



In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import logging
logging.getLogger('mmdet').setLevel(logging.WARNING)
# Now INFO and DEBUG from some_lib won’t show up.



class MultiViewSkeletonClassifier(nn.Module):
    """
    PyTorch model for multi-view skeleton fusion and classification.

    Inputs:
        - Four skeleton feature tensors, each of shape (batch_size, seq_len=20, feat_dim=512).
    Output:
        - Logits for classification into num_classes.
    """
    def __init__(self, skeleton_model, feat_dim=512, seq_len=20, num_views=4, hidden_dim=512, num_classes=10):
        super(MultiViewSkeletonClassifier, self).__init__()
        self.num_views = num_views
        self.seq_len = seq_len
        self.feat_dim = feat_dim
        self.skeleton_model = skeleton_model
        
        # Define the attention head for per-view summary
        self.attn_head = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(inplace=True),
            nn.Linear(hidden_dim // 2, 1)
        )
        
        

        # Learnable fusion weights for each view (softmax-normalized)
        self.view_weights = nn.Parameter(torch.ones(num_views))

        # Project concatenated features to hidden dimension
        self.fusion_proj = nn.Linear(feat_dim, hidden_dim)

        # Temporal convolutional layers
        self.conv1 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = nn.Conv1d(hidden_dim, hidden_dim, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(hidden_dim)

        # Classification head
        self.fc = nn.Sequential(
            nn.Linear(hidden_dim * seq_len, hidden_dim),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(hidden_dim, num_classes)
        )

    def forward(self, batch):
        """
        Args:
            views: list or tuple of length num_views, each tensor of shape (B, T, D)
        Returns:
            logits: tensor of shape (B, num_classes)
        """
        # Stack views: (V, B, T, D) -> (B, V, T, D)
        
            
        x = torch.stack(batch).to(device='cuda:0')  # (B, V, T, D)
        B, V, T, D = x.shape
        
        # per‐view summary → score → masked softmax → fusion
        summary = x.mean(dim=2)              # (B, V, hidden)
        raw_w   = self.attn_head(summary).squeeze(-1) # (B, V)
        w       = F.softmax(raw_w)    # (B, V)
        fused   = (w.view(B,V,1,1) * x).sum(dim=1)


        # Project features
        fused = self.fusion_proj(fused)  # (B, T, hidden_dim)
        fused = F.relu(fused)

        # Prepare for temporal conv: (B, hidden_dim, T)
        fused = fused.permute(0, 2, 1)

        # Temporal convolutional block
        out = F.relu(self.bn1(self.conv1(fused)))
        out = F.relu(self.bn2(self.conv2(out)))

        # Flatten temporal features
        out = out.view(B, -1)

        # Classification head
        logits = self.fc(out)
        
        # Apply softmax to get probabilities
        logits = F.softmax(logits, dim=1)
        return logits


if __name__ == "__main__":
    # Example usage
    B, T, D = 8, 20, 512
    num_views = 4
    num_classes = 4
    multiview_model = MultiViewSkeletonClassifier(skeleton_model=skeleton_model, feat_dim=512, seq_len=T, num_views=num_views, hidden_dim=D, num_classes=num_classes).to('cuda:0')
    # Create dummy inputs
    views = [torch.randn(B, T, D) for _ in range(num_views)]
    
    batch = process_skeletons(([data, data]), skeleton_model, pose_inference, num_windows=2)
    results = multiview_model(batch)  # Forward pass
    
    print("Input shapes:", [view.device for view in views])  # expected (8, 20, 512)
    print(results)
     # expected (8, num_classes)


Length of video: 255
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 10.0 task/s, elapsed: 13s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 18.3 task/s, elapsed: 7s, ETA:     0s
128
Length of video: 255
05/13 18:21:59 - mmengine - WARNING - The current default scope "mmpose" is not "mmdet", `init_default_scope` will force set the currentdefault scope to "mmdet".
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 9.4 task/s, elapsed: 14s, ETA:     0s
05/13 18:22:13 - mmengine - WARNING - The current default scope "mmdet" is not "mmpose", `init_default_scope` will force set the currentdefault scope to "mmpose".
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 16.0 task/s, elapsed: 8s, ETA:     0s
128
Length of video: 255
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 9.4 task/s, elapsed: 14s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 16.3 task/s, elapsed: 8s, ETA:     0s
128
Length of video: 255
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 9.6 task/s, elapsed: 13s, ETA:     0s
[>>>>>>>>>>>>>>>>>>

/tmp/ipykernel_348406/1065641017.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  w       = F.softmax(raw_w)    # (B, V)


In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_epochs    = 30
learning_rate = 0.001
batch_size    = 8
num_classes   = 4

# Model
multiview_model = MultiViewSkeletonClassifier(
    skeleton_model=skeleton_model,
    feat_dim=512,
    seq_len=20,
    num_views=4,
    hidden_dim=512,
    num_classes=num_classes
).to(device)
model = multiview_model

# Loss and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer)


# Dataloaders
train_loader = DataLoader(
    train_dataset,
    collate_fn=collate_objects,
    batch_size=batch_size,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,  # assuming you have a separate validation dataset
    collate_fn=collate_objects,
    batch_size=batch_size,
    shuffle=False
)

# Training + Validation loop
for epoch in range(num_epochs):
    # ---- TRAIN ----
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for views, labels in tqdm(train_loader, desc=f"Train Epoch {epoch+1}/{num_epochs}"):
        # if views is a list of tensors; otherwise .to(device)
        labels = labels.to(device)

        # forward + backward
        batch = process_skeletons(views, skeleton_model, pose_inference, num_windows = 10, window_size=128, sampling_rate=5)
        outputs = model(batch)
        loss    = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # stats
        running_loss += loss.item()
        _, preds = outputs.max(1)
        total   += labels.size(0)
        correct += preds.eq(labels).sum().item()
        
    
    train_loss = running_loss / len(train_loader)
    train_acc  = 100. * correct / total

    # ---- VALIDATION ----
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total   = 0

    with torch.no_grad():
        for views, labels in tqdm(val_loader, desc=f"Val   Epoch {epoch+1}/{num_epochs}"):
            
            labels = labels.to(device)
            batch = process_skeletons(views, skeleton_model, pose_inference, num_windows = 10, window_size=128, sampling_rate=5)
            

            outputs = model(batch)
            loss    = criterion(outputs, labels)

            val_loss += loss.item()
            _, preds = outputs.max(1)
            val_total   += labels.size(0)
            val_correct += preds.eq(labels).sum().item()

    val_loss_epoch = val_loss / len(val_loader)
    val_acc        = 100. * val_correct / val_total
    scheduler.step(val_loss_epoch)
    # ---- EPOCH SUMMARY ----
    print(
        f"Epoch {epoch+1}/{num_epochs} | "
        f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
        f"Val   Loss: {val_loss_epoch:.4f}, Val   Acc: {val_acc:.2f}%"
    )
    torch.save(model.state_dict(), f"multiview_model_epoch_{epoch+1}.pth")
print("Training complete.")



Train Epoch 1/30:   0%|          | 0/71 [00:00<?, ?it/s]

Length of video: 1076.0
05/13 18:04:52 - mmengine - WARNING - The current default scope "mmaction" is not "mmdet", `init_default_scope` will force set the currentdefault scope to "mmdet".
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 9.8 task/s, elapsed: 13s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 14.7 task/s, elapsed: 9s, ETA:     0s
128
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 9.8 task/s, elapsed: 13s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 15.7 task/s, elapsed: 8s, ETA:     0s
256
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 103/103, 10.5 task/s, elapsed: 10s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 103/103, 13.4 task/s, elapsed: 8s, ETA:     0s
359
Length of video: 1076.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 9.9 task/s, elapsed: 13s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>          ] 89/128, 16.9 task/s, elapsed: 5s, ETA:     2s

Train Epoch 1/30:   0%|          | 0/71 [01:22<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
import json
# Testing loop
test_loader = DataLoader(
    val_dataset,
    collate_fn=collate_objects,
    batch_size=2,
    shuffle=False
)

# Load the trained model

model.eval()

test_loss = 0.0
test_correct = 0
test_total = 0

with torch.no_grad():
    videos = []
    ego_model_predictions = []
    exo_model_predictions = []
    for views, labels in tqdm(test_loader, desc="Testing"):
        it_exo_model_predictions = []
        labels = labels.to(device)

        videos += [views[i]['samples']['ego'].split("/")[-5] for i in range(len(views))]
        batch = process_skeletons(views, skeleton_model, pose_inference, window_size=128, sampling_rate=64)
        
        outputs = model(batch)
        
        exo_model_predictions += [[outputs[i].cpu().tolist()] * 4 for i in range(len(outputs))]
        loss = criterion(outputs, labels)

        test_loss += loss.item()
        _, preds = outputs.max(1)
        test_total += labels.size(0)
        test_correct += preds.eq(labels).sum().item()

test_loss_epoch = test_loss / len(test_loader)
test_acc = 100. * test_correct / test_total

results = {"videos": videos, "exo_model_predictions": exo_model_predictions}

with open("results.json", "w") as f:
    json.dump(results, f, indent=2) 

# ---- TEST SUMMARY ----
print(
    f"Test Loss: {test_loss_epoch:.4f}, Test Acc: {test_acc:.2f}%"
)

Testing:   0%|          | 0/74 [00:00<?, ?it/s]

Length of video: 8385.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 9.0 task/s, elapsed: 14s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 16.8 task/s, elapsed: 8s, ETA:     0s
128
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4/4, 9.2 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4/4, 16.3 task/s, elapsed: 0s, ETA:     0s
132
Length of video: 8385.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 9.1 task/s, elapsed: 14s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 17.5 task/s, elapsed: 7s, ETA:     0s
128
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4/4, 9.1 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4/4, 16.7 task/s, elapsed: 0s, ETA:     0s
132
Length of video: 8385.0
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 9.2 task/s, elapsed: 14s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 128/128, 17.6 task/s, elapsed: 7s, ETA:     0s
128
[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 4/4, 9.4 task/s, elapsed: 0s, ETA:     0s
[>>>>>>>>>>>>>>>>>>>>>>>>>>>

/tmp/ipykernel_346772/4071208005.py:71: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  w       = F.softmax(raw_w)    # (B, V)
/home/thibault/miniconda3/envs/ego-challenge/lib/python3.12/site-packages/debugpy/_vendored/pydevd/pydev_ipython/matplotlibtools.py:59: MatplotlibDeprecationWarning: The interactive_bk attribute was deprecated in Matplotlib 3.9 and will be removed in 3.11. Use ``matplotlib.backends.backend_registry.list_builtin(matplotlib.backends.BackendFilter.INTERACTIVE)`` instead.
  from matplotlib.rcsetup import interactive_bk, non_interactive_bk  # @UnresolvedImport
/home/thibault/miniconda3/envs/ego-challenge/lib/python3.12/site-packages/debugpy/_vendored/pydevd/pydev_ipython/matplotlibtools.py:59: MatplotlibDeprecationWarning: The non_interactive_bk attribute was deprecated in Matplotlib 3.9 and will be removed in 3.11. Use ``matplotlib.backends.backend_registry.list_builtin(matplotlib.backends.B

KeyboardInterrupt: 